### Warm and Cold Events in the MITgcm runs forced by PACE
This document is made for the detection, characterisation and study of warm and cold events in MITgcm runs by Kaitlin.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr
import unixFunctionsMITgcm_final as fc

#### Step 1: Definition and Detection
In order to find a warm or cold event in the MITgcm data, we need a detection limit.